In [1]:
# --------------------
#  라이브러리 및 데이터 준비
# --------------------
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet # ElasticNet 회귀 모델
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 예시 데이터 생성 (특성 10개, 샘플 50개)
rng = np.random.RandomState(42)
X = rng.rand(50, 10) * 10
# 타겟값 생성: Feature0, Feature5만 영향을 주고, Feature1, Feature2는 상관관계가 높다고 가정
X[:, 1] = X[:, 2] * 0.95 + rng.rand(50) * 0.5 # 다중 공선성 추가
y = 3 * X[:, 0] - 2 * X[:, 5] + 1 + rng.randn(50) * 5 

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("--- 학습 데이터 크기 ---")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")

--- 학습 데이터 크기 ---
X_train: (35, 10), y_train: (35,)


In [2]:
# --------------------
#  2단계: 데이터 스케일링 (표준화)
# --------------------
scaler = StandardScaler()
# 학습 데이터에 fit 후 transform
X_train_scaled = scaler.fit_transform(X_train)
# 테스트 데이터는 fit 없이 transform만
X_test_scaled = scaler.transform(X_test)

print("\n--- 2단계: 스케일링 완료 ---")


--- 2단계: 스케일링 완료 ---


In [3]:
# --------------------
#  3단계: ElasticNet 모델 학습 및 계수 확인
# --------------------
# alpha=0.5: 전체 규제 강도 설정 (λ)
# l1_ratio=0.7: L1 규제의 비율을 70%로 설정 (나머지 30%는 L2)
enet_model = ElasticNet(alpha=0.5, l1_ratio=0.7, random_state=42) 

# 모델 학습
enet_model.fit(X_train_scaled, y_train)

# 학습된 회귀 계수 (가중치) 확인
coefficients = enet_model.coef_

print("\n--- 3단계: 학습된 회귀 계수 (β) ---")
# ElasticNet은 0에 가까운 계수와 완전히 0인 계수(Lasso의 영향)가 모두 나타날 수 있습니다.
print(coefficients)


--- 3단계: 학습된 회귀 계수 (β) ---
[ 8.08511169 -0.65341159 -0.16181936 -0.37043579 -0.72804464 -4.76869193
 -0.          0.28498394 -1.09494374  0.        ]


In [4]:
# --------------------
#  4단계: 예측 및 성능 평가
# --------------------
from sklearn.metrics import mean_squared_error

# 예측 수행
y_pred = enet_model.predict(X_test_scaled)

# 성능 평가 (MSE 사용)
mse = mean_squared_error(y_test, y_pred)

print(f"\n--- 4단계: 모델 성능 평가 ---")
print(f"테스트 데이터 MSE (평균 제곱 오차): {mse:.4f}")


--- 4단계: 모델 성능 평가 ---
테스트 데이터 MSE (평균 제곱 오차): 34.9621
